In [1]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib


def write_rcbox(series, tree, root):
    for centre in root.iter('centre'):
        centre.text = str(series['x_centro']) + ' ' + str(series['y_centro']) + ' ' + str(series['z_centro'])
    for widthHeightDepth in root.iter('widthHeightDepth'):
        widthHeightDepth.text = str(series['width']) + ' ' + str(series['height']) + ' ' + str(series['depth'])
    tree.write(series['rcboxoutpath'])
def createTileFrame(geodf):
    geo = geodf.geometry
    #geo_clean = geo[~(geo.is_empty | geo.isna())]
    geo_union = geo.unary_union
    geo_envelope = geo_union.envelope
    return geo_envelope
def createBuffer(grid_df):
    grid_geom = grid_df.geometry
    grid_df['buffer'] = grid_geom.geometry.buffer(50)  
    return grid_df
def getCameralist(series, geodf):
    
    cameraswithinlist = geodf.geometry.within(series.geometry)
    cameraswithin = geodf.loc[cameraswithinlist]
    series['cameralist']=list(cameraswithin['FileName'])
    return series
def getNeighbourlist(series, grid_df):
    neighbourlist = grid_df.geometry.intersects(series.geometry)
    neighbour = grid_df.loc[neighbourlist]
    series['neighbours'] = list(neighbour['GridId'])
    return series
def createGrid(geodf_envelope, length, width):
    xmin,ymin,xmax,ymax = geodf_envelope.total_bounds
    cols = list(range(int(np.floor(xmin)), int(np.ceil(xmax)), width))
    rows = list(range(int(np.floor(ymin)), int(np.ceil(ymax)), length))
    #rows.reverse()
    grid_df = gpd.GeoDataFrame()
    for ix,x in enumerate(cols): 
        for iy,y in enumerate(rows):
            grid={}
            grid['GridId']= str(int(ix)) + '_' + str(int(iy))
            grid['geometry'] = Polygon([(x,y), (x+width, y), (x+width, y+length), (x, y+length)])
            grid_df = grid_df.append(grid, True)
    return grid_df

def create_rcboxoutpath(series, outputfolder):
    series['rcboxoutpath'] = os.path.join(outputfolder, 'tile3D_' + str(series['id']) + '.rcbox')
    return series

def load_xml(name):
    tree = ET.parse(name)
    root = tree.getroot()
    return tree, root

In [5]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = '*.dng')
scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_PNG', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
scandf2=pd.DataFrame()
for index,scan in scandf.iterrows():
    scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0], outputfolderpath=scan['RToutputfolder'], axis=1) 
    scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
    scan = RClib.createRCproject(scan)
    scan['commandlist_align'] = []
    scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
    scan['commandlist_align'].append('-detectMarkers')
    scan['commandlist_align'].append('-align')
    scan['commandlist_align'].append('-save')
    scan = RClib.executeRCCMDuseRCproject(scan, commandlistfield='commandlist_align' )


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TypeError: can only concatenate list (not "str") to list

In [3]:
%load_ext autoreload
%autoreload 2
tree, root = load_xml(template)
df = df.apply(create_rcboxoutpath, outputfolder=outputfolder, axis=1)
df.apply(write_rcbox, tree=tree, root=root, axis=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


NameError: name 'template' is not defined